In [1]:
import os
os.chdir('/cluster/home/bdayan/girgs/benji_src/notebooks/')

import sys
sys.path.append('../')
sys.path.append('../../nemo-eva/src/')

import os
os.getcwd()

import feature_extractor

from benji_girgs import utils, generation, points, fitting, mcmc
import os
import glob
import pandas as pd
import networkit

import networkit as nk

from girg_sampling import girgs

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import matplotlib.pyplot as plt

import do_feature_extract
import networkx as nx

import pickle


../benji_girgs/points.py:206: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (n, d == other.shape)


/cluster/scratch/bdayan/GIRG_data/socfb-Brandeis99.SpaceOne
/cluster/scratch/bdayan/GIRG_data/socfb-Swarthmore42.SpaceOne
/cluster/scratch/bdayan/GIRG_data/socfb-Georgetown15.SpaceOne
/cluster/scratch/bdayan/GIRG_data/socfb-UC64.SpaceOne
/cluster/scratch/bdayan/GIRG_data/socfb-Bowdoin47.SpaceOne
/cluster/scratch/bdayan/GIRG_data/socfb-Middlebury45.SpaceOne
/cluster/scratch/bdayan/GIRG_data/socfb-MIT.SpaceOne
/cluster/scratch/bdayan/GIRG_data/socfb-WashU32.SpaceOne
/cluster/scratch/bdayan/GIRG_data/socfb-William77.SpaceOne
/cluster/scratch/bdayan/GIRG_data/socfb-Pepperdine86.SpaceOne
/cluster/scratch/bdayan/GIRG_data/socfb-Emory27.SpaceOne
/cluster/scratch/bdayan/GIRG_data/socfb-Lehigh96.SpaceOne
/cluster/scratch/bdayan/GIRG_data/socfb-Mich67.SpaceOne
/cluster/scratch/bdayan/GIRG_data/socfb-Princeton12.SpaceOne
/cluster/scratch/bdayan/GIRG_data/socfb-Rice31.SpaceOne
/cluster/scratch/bdayan/GIRG_data/socfb-Columbia2.SpaceOne
/cluster/scratch/bdayan/GIRG_data/socfb-Williams40.SpaceOne
/cl

In [2]:
import importlib
importlib.reload(generation)

<module 'benji_girgs.generation' from '../benji_girgs/generation.py'>

In [3]:
n = 2000
d=3
tau=2.5
alpha=1.5
desiredAvgDegree= 60.0

g, edges, weights, pts, const = generation.generate_GIRG_nk(
        n, d, tau, alpha, desiredAvgDegree=desiredAvgDegree, points_type=points.PointsTorus2)

../benji_girgs/generation.py:161: RuntimeWarning: divide by zero encountered in divide
  p_uv = np.divide(outer, dists**d)


In [4]:
nk.overview(g)

Network Properties:
nodes, edges			2000, 59807
directed?			False
weighted?			False
isolated nodes			0
self-loops			0
density				0.029918
clustering coefficient		0.408550
min/max/avg degree		12, 1305, 59.807000
degree assortativity		-0.108320
number of connected components	1
size of largest component	2000 (100.00 %)


In [5]:
g2, _, weights, pts, const2 = generation.generate_GIRG_nk(
        n, d, tau, alpha, desiredAvgDegree=desiredAvgDegree, points_type=points.PointsMCD, weights=weights, pts=None, c_implementation=False)

print(const2)

# g2, _, _, _, const2 = generation.generate_GIRG_nk(
#         n, d, tau, alpha, desiredAvgDegree=desiredAvgDegree, points_type=points.PointsMCD, weights=weights, pts=pts, c_implementation=True)

1.1158475793617662


In [6]:
nk.overview(g2)

Network Properties:
nodes, edges			2000, 60263
directed?			False
weighted?			False
isolated nodes			0
self-loops			0
density				0.030147
clustering coefficient		0.278165
min/max/avg degree		14, 1318, 60.263000
degree assortativity		-0.107998
number of connected components	1
size of largest component	2000 (100.00 %)


In [8]:
g_out, graphs = generation.quick_MCD_algo_test(n, d, tau, alpha, desiredAvgDegree=desiredAvgDegree, weights=weights, pts=pts)

generating i=1 out of 3
generating i=2 out of 3
generating i=3 out of 3
checking edges of i=1 out of 3


25652it [00:02, 11786.54it/s]


checking edges of i=2 out of 3


25654it [00:01, 13627.94it/s]


checking edges of i=3 out of 3


25708it [00:01, 14492.45it/s]


In [9]:
nk.overview(g_out)

Network Properties:
nodes, edges			2000, 59926
directed?			False
weighted?			False
isolated nodes			0
self-loops			0
density				0.029978
clustering coefficient		0.273704
min/max/avg degree		11, 1296, 59.926000
degree assortativity		-0.106917
number of connected components	1
size of largest component	2000 (100.00 %)


In [10]:
def adj_mat(g):
    gnx = nk.nxadapter.nk2nx(g)
    return nx.adjacency_matrix(gnx).todense()

a, b = adj_mat(g2), adj_mat(g_out)


In [11]:
mcmc.CM(a, b)

(array([[  70804,   49722],
        [  49048, 3830426]]),
 0.5874583077510247,
 0.40923806027433834)

In [7]:
const
c = generation.const_conversion(const, alpha, d=d, true_volume=True)
print(f'c: {c}')

i = 0
pts_i = pts[:, i:i+1]
weight_normaliser = np.sqrt(np.sum(weights))
pts_i.true_d = d
adj_mat = generation.generateEdges(weights / weight_normaliser, pts_i, alpha, c=c, other_d=d)
g_i = generation.edge_list_to_nk_graph(zip(*np.nonzero(adj_mat)), n)
print(g_i.numberOfEdges())

1.1158475793617662

c: 26.67118390288009
25656


In [29]:
i = 0
weight_normaliser = np.sqrt(np.sum(weights))
adj_mat = generation.generateEdges(weights / weight_normaliser, pts[:, i:i+1], alpha, c=c, other_d=d)
g = generation.edge_list_to_nk_graph(zip(*np.nonzero(adj_mat)), n)

In [31]:
nk.overview(g)

Network Properties:
nodes, edges			2000, 495380
directed?			False
weighted?			False
isolated nodes			0
self-loops			0
density				0.247814
clustering coefficient		0.708590
min/max/avg degree		329, 1862, 495.380000
degree assortativity		0.003788
number of connected components	1
size of largest component	2000 (100.00 %)


In [32]:
w = weights / weight_normaliser
pts_i = pts[:, i:i+1]


In [43]:
const = girgs.scaleWeights(weights, desiredAvgDegree, d, alpha)
c = generation.const_conversion(const, alpha, d=d, true_volume=True)
print(const)
print(c)
probs = generation.get_probs(w, pts, alpha, c=c)
print(probs.sum()/2)

1.1385364765434698
27.488776118627037
61053.49814221759


In [45]:
const = girgs.scaleWeights(weights, desiredAvgDegree, 1, alpha)
c1 = generation.const_conversion(const, alpha, d=1, true_volume=True)
print(const)
print(c1)
probs = generation.get_probs(w, pts_i, alpha, c=c1)
print(probs.sum()/2)

4.554145906173879
27.488776118627037
61287.19852395212


In [46]:
probs = generation.get_probs(w, pts_i, alpha, c=c1, other_d=d)
print(probs.sum()/2)

496148.79607919103


In [37]:
probs.sum()/2

PointsMCD(61287.19852395)

In [16]:
sum([x.numberOfEdges() for x in graphs])
g2.numberOfEdges()

73451

89841

In [11]:
graphs[0].numberOfEdges()

168813

In [12]:
desiredAvgDegree

60.0

In [48]:
a1, a2, a3 = graphs
utils.avg_degree(a1)
utils.avg_degree(a2)

6.352

6.455333333333333

In [49]:
for edge in a1.iterEdges():
    print(edge)

(0, 2148)
(0, 2423)
(0, 1325)
(0, 2533)
(0, 2188)
(0, 278)
(0, 270)
(0, 2861)
(0, 513)
(0, 2543)
(0, 2880)
(0, 1144)
(0, 2376)
(0, 589)
(0, 1050)
(0, 1322)
(0, 2988)
(1, 2148)
(1, 2399)
(1, 1593)
(1, 1034)
(2, 2148)
(2, 1224)
(2, 2103)
(2, 2709)
(2, 904)
(2, 243)
(3, 2148)
(3, 770)
(3, 1459)
(3, 231)
(3, 333)
(3, 962)
(3, 1105)
(3, 1749)
(3, 1752)
(3, 854)
(4, 2148)
(4, 2310)
(4, 2621)
(5, 91)
(6, 396)
(7, 2148)
(7, 2527)
(7, 650)
(7, 1341)
(8, 934)
(9, 2148)
(9, 2664)
(9, 2007)
(9, 1478)
(9, 2221)
(9, 424)
(9, 507)
(9, 2170)
(10, 2148)
(10, 1269)
(10, 1382)
(10, 2919)
(10, 1875)
(10, 343)
(11, 2148)
(11, 694)
(11, 2425)
(11, 2940)
(11, 1013)
(11, 1907)
(11, 1202)
(11, 1900)
(12, 2005)
(13, 2148)
(13, 558)
(13, 441)
(14, 2148)
(14, 770)
(14, 1166)
(15, 2148)
(16, 2148)
(16, 1608)
(16, 1459)
(16, 841)
(16, 1587)
(16, 2306)
(16, 1240)
(16, 2558)
(17, 2148)
(17, 1526)
(17, 2343)
(18, 2148)
(18, 2934)
(18, 1937)
(18, 1733)
(18, 675)
(19, 2915)
(20, 2148)
(21, 2148)
(21, 2354)
(21, 1975)
(2

In [51]:
foo = generation.generatePositions(5, 3)
foo = points.PointsTorus2(foo)
bar = girgs.generatePositions(5, 3)

foo
bar

PointsTorus2([[0.1604118 , 0.7884019 , 0.82092185],
              [0.95728   , 0.21154488, 0.39747297],
              [0.30266202, 0.09793944, 0.77269729],
              [0.59887567, 0.56346661, 0.02219453],
              [0.44663974, 0.95149238, 0.98857793]])

[[0.7341851167561663, 0.32559338191363196, 0.6522098505767389],
 [0.9595753267370022, 0.7547941147962371, 0.24825267139595916],
 [0.5201790732423792, 0.04188390800235924, 0.9893345681474501],
 [0.8010247749572212, 0.25851298279954626, 0.24015400482675314],
 [0.8567441139413051, 0.44952211289790994, 0.42225343215625505]]

In [60]:
np.argmin([foo[0, i:i+1].dist(foo[1, i:i+1]) for i in [0,1,2]])

0

In [61]:
foo[0]
foo[1]
[foo[0, i:i+1].dist(foo[1, i:i+1]) for i in [0,1,2]]

PointsTorus2([0.1604118 , 0.7884019 , 0.82092185])

PointsTorus2([0.95728   , 0.21154488, 0.39747297])

[PointsTorus2(0.40625), PointsTorus2(0.84570312), PointsTorus2(0.84667969)]

In [63]:
2* (0.8209 - 0.3974)

0.847

In [59]:
g, edges, weights, pts, const = generation.cgirg_gen(3000, 3, 2.5, 1.5, 20.0)

In [17]:
edges[:10]

[(1701, 1403),
 (1701, 990),
 (1701, 2699),
 (1701, 1268),
 (1701, 2801),
 (1701, 949),
 (1701, 572),
 (1701, 1922),
 (1158, 1403),
 (1158, 990)]

In [19]:
edges[1000:1010]

[(2510, 23),
 (2510, 2511),
 (2548, 1439),
 (2510, 2601),
 (2510, 2929),
 (2510, 1989),
 (2510, 1949),
 (317, 398),
 (1064, 1214),
 (1064, 89)]

In [23]:
g.degree(1700)

16

In [26]:
g.hasEdge(2510, 1950)

False